In [1]:
#import pandas as pd
import polars as pl
import numpy as np
from datetime import datetime
import json
import os
import math
import pandas as pd
from sklearn.preprocessing import PowerTransformer, StandardScaler, RobustScaler
import time

import warnings

import pandas as pd

warnings.filterwarnings('ignore')

fase = '02_normaliza'

In [2]:
with open('gen_config.json', 'r') as file:
    gen_config =json.load(file)

In [3]:
folder = gen_config['folder']

#entradas
path_group = gen_config['path_group']
path_prod_stats = gen_config['path_prod_stats']

#salidas
path_norm = gen_config['path_norm']
#path_transform_stats = gen_config['path_transform_stats']

#variables
var_escalado = gen_config['var_escalado']
var_withmean = gen_config['var_withmean']

print(f"{'COMIENZA':-^100}")
print(f"{fase:-^100}")

----------------------------------------------COMIENZA----------------------------------------------
--------------------------------------------02_normaliza--------------------------------------------


In [4]:
df_norm = pl.read_parquet(f"{folder}/{path_group}")
prod_stats = pl.read_parquet(f'{folder}/{path_prod_stats}')

In [5]:
df_norm = df_norm.to_pandas()
prod_stats = prod_stats.to_pandas()

In [6]:
grouped = df_norm.groupby(['product_id', 'customer_id'])

# Standard Scaler

In [7]:
def scale_standard(group):
    scaler = StandardScaler(with_mean=var_withmean)
    group['tn_standard'] = scaler.fit_transform(group[['tn']])
    # Store the scaler in a dictionary with a tuple key
    standard_scalers[(group['product_id'].iloc[0], group['customer_id'].iloc[0])] = scaler
    return group

In [8]:
standard_scalers = {}
df_standard = grouped.apply(scale_standard)
df_standard = df_standard.reset_index(drop=True)

In [ ]:
scaler_data = []
for (product_id, client_id), scaler in standard_scalers.items():
    scaler_data.append({
        'product_id': product_id,
        'client_id': client_id,
        'standard_scaler_mean': scaler.mean_[0],
        'standard_scaler_scale': scaler.scale_[0]
    })

scalers_standard_stats = pd.DataFrame(scaler_data)

# Robust Scaler

In [ ]:
def scale_robust(group):
    scaler = RobustScaler(with_centering=var_withmean)
    group['tn_robust'] = scaler.fit_transform(group[['tn']])
    # Store the scaler in a dictionary with a tuple key
    robust_scalers[(group['product_id'].iloc[0], group['customer_id'].iloc[0])] = scaler
    return group

In [ ]:
robust_scalers = {}
df_robust = grouped.apply(scale_robust)
df_robust = df_robust.reset_index(drop=True)

In [ ]:
scaler_data = []
for (product_id, client_id), scaler in robust_scalers.items():
    scaler_data.append({
        'product_id': product,
        'client_id': client,
        'robust_scaler_center': scaler.center_[0],
        'robust_scaler_scale': scaler.scale_[0]
    })

scalers_robust_stats = pd.DataFrame(scaler_data)

# DF finales

In [ ]:
df_norm = df_norm.merge(df_standard[['product_id', 'customer_id', 'tn_standard']], on=['product_id', 'customer_id', 'periodo'], how='left')
df_norm = df_norm.merge(  df_robust[['product_id', 'customer_id', 'tn_robust']]  , on=['product_id', 'customer_id', 'periodo'], how='left')
df_norm = pl.from_pandas(df_norm)

,product_id,customer_id,periodo,tn,tn_standard
0,20001,10001,201701,99.43861,0.951113
1,20001,10001,201702,198.84365,1.901905
2,20001,10001,201703,92.46537,0.884415
3,20001,10001,201704,13.29728,0.127186
4,20001,10001,201705,101.00563,0.966101
...,...,...,...,...,...
5303550,21276,10550,201908,0.00000,0.000000
5303551,21276,10550,201909,0.00075,0.675225
5303552,21276,10550,201910,0.00000,0.000000
5303553,21276,10550,201911,0.00371,3.340115


In [ ]:
prod_stats = prod_stats.merge(scalers_standard_stats, on=['product_id', 'client_id'], how='left')
prod_stats = prod_stats.merge(scalers_robust_stats, on=['product_id', 'client_id'], how='left')
prod_stats = pl.from_pandas(prod_stats)

In [ ]:
# grouped = df_scaled.groupby(['product_id', 'customer_id'])
# df_unscaled = grouped.apply(inverse_scale_sales)
# df_unscaled = df_unscaled.reset_index(drop=True)

# df_unscaled

In [ ]:
# def inverse_scale_standard(group):
#     scaler = standard_scalers[(group['product_id'].iloc[0], group['customer_id'].iloc[0])]
#     group['tn_inverse'] = scaler.inverse_transform(group[['tn_standard']])
#     return group

In [ ]:
# def inverse_scale_robust(group):
#     scaler = robust_scalers[(group['product_id'].iloc[0], group['customer_id'].iloc[0])]
#     group['tn_inverse'] = scaler.inverse_transform(group[['tn_standard']])
#     return group

In [ ]:
# def create_scaler_from_row(row):
#     scaler = StandardScaler(with_mean=var_withmean)
#     scaler.mean_ = [row['scaler_mean']]
#     scaler.scale_ = [row['scaler_scale']]
#     scaler.var_ = [row['scaler_scale']**2]
#     return scaler

In [ ]:
prod_stats.write_parquet(f'{folder}/{path_prod_stats}')
df_norm.write_parquet(f'{folder}/{path_norm}')

In [ ]:
print(f"{fase:-^100}")
print(f"{'FINALIZA':-^100}\n\n\n")

--------------------------------------------02_normaliza--------------------------------------------
----------------------------------------------FINALIZA----------------------------------------------



